In [1]:
import tqdm 
import pandas as pd
import akshare as ak

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True

def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
    
    
df = ak.stock_zh_a_spot_em()
df = df.rename(columns={"代码":"code","名称":"name"})
df = df[df.apply(filter_shsz,axis=1)]
df["code"] = df["code"].apply(reform_shsz)
cdf = df[["code","name"]]
lst = []
hdf = pd.read_csv("stock_hist.csv")
hs = set(hdf["code"].drop_duplicates())
cdf = cdf[~cdf.code.isin(hs)]
lst = [hdf]
bar = tqdm.tqdm(total =cdf.shape[0])

for i,(idx,v)in enumerate(cdf.iterrows()):
    bar.update()
    if i>1000:
        break
    try:
        tdf = ak.stock_zh_a_cdr_daily(symbol=v["code"], start_date="20241116")[["date","open","high","low","close","volume","amount"]]
        tdf[["code","name"]]=(v["code"],v["name"])
        lst.append(tdf)
    except:
        print(v)
if len(lst)>1:
    hdf = pd.concat(lst)
    hdf['code'] = hdf['code'].astype(str)
    hdf.to_csv("stock_hist.csv",index=False)


/Users/john/anaconda3/lib/python3.11/site-packages/akshare/__init__.py:2977: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.2.0 及以上版本！
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'stock_hist.csv'

In [12]:
import akshare as ak
import datetime

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True


def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
today, bias = datetime.datetime.now(), 0
cdate = today+datetime.timedelta(days=-bias)
cdate = cdate.strftime('%Y-%m-%d')
cdf = ak.stock_zh_a_spot_em()
cdf = cdf.rename(columns={"代码":"code","名称":"name"})
cdf = cdf[cdf.apply(filter_shsz,axis=1)]
cdf["code"] = cdf["code"].apply(reform_shsz)
cdf = cdf.rename(columns={"今开":"open","最低":"low","最高":"high","最新价":"close",
                         "成交量":"volume","成交额":"amount"})
cdf["date"]=cdate
frac = 1
cdf["volume"] = cdf["volume"]*100*frac
cdf["amount"] = cdf["amount"]*frac
cdf = cdf[["date","code","name","open","low","high","close","volume","amount"]]
# hdf = pd.read_csv("stock_hist.csv")
# df = pd.concat([hdf,cdf])
# cdf.to_csv("stock_today.csv",index=False)
cdf.to_csv("stock_today.csv",index=False)
cdf.head()

,date,code,name,open,low,high,close,volume,amount
0,2025-01-03,sh603072,N天和,40.00,38.01,100.00,48.00,50782000.0,2.549723e+09
1,2025-01-03,sz301581,N黄山谷,80.00,69.10,99.99,70.01,15753400.0,1.301801e+09
4,2025-01-03,sz300615,欣天科技,15.27,14.33,16.55,16.55,48571000.0,7.560458e+08
5,2025-01-03,sz301568,思泰克,34.20,34.20,40.32,40.32,19704500.0,7.788002e+08
8,2025-01-03,sz300630,普利制药,6.81,6.55,7.87,7.72,69824800.0,5.126802e+08


In [13]:
import pandas as pd

def save_dt():
    def get_weekday(cdate):
        holidays = set(["2025-01-01"])
        for i in range(1,5):
            ttm = today+datetime.timedelta(days=-bias-i)
            if ttm.weekday() <5 and ttm.strftime("%Y-%m-%d") not in holidays:
                return ttm

    today = datetime.datetime.now()
    cdate = today+datetime.timedelta(days=-bias)
    pdate = get_weekday(cdate)
    cdate,pdate = cdate.strftime('%Y-%m-%d'),pdate.strftime('%Y-%m-%d')
    df = pd.read_csv(f"daily/stock_{pdate}.csv")
    cdf = pd.read_csv("stock_today.csv")
    df = pd.concat([df,cdf])
    df.to_csv(f"daily/stock_{cdate}.csv",index=False)
    cdf.to_csv(f"daily/stock_daily_{cdate}.csv",index=False)

def clean_data(pattern="stock_\d{4}-\d{2}-\d{2}.csv"):
    import os, re
    lst = []
    for r,d, files in os.walk("./daily"):
        for f in files:
            if re.match(pattern,f):
                lst.append(os.path.join(r,f))
    for f in sorted(lst)[:-2]:
        os.remove(f)

save_dt()
clean_data()

In [46]:
import akshare as ak
sh = ak.stock_zh_index_daily(symbol="sh000001")[-100:]
sz = ak.stock_zh_index_daily(symbol="sz399001")[-100:]
def reprocess(df):
    df = df.sort_values(by="date",ascending=True).reset_index(drop=True)
    changes = []
    for i, r in df.iterrows():
        change = (df.iloc[i]["close"] - df.iloc[i-1]["close"])/df.iloc[i-1]["close"] if i else 0.0
        changes.append(change*100)
    df["change"] = changes
    return df
sh = reprocess(sh)
sz = reprocess(sz)
sh["code"] = "sh000001"
sz["code"] = "sz399001"

In [60]:
import tqdm
import numpy as np
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
cdate = "2025-01-02"
dz = {"sh":sh["change"].values.reshape((-1,1)), "sz":sz["change"].values.reshape((-1,1))}
df = pd.read_csv(f"./daily/stock_{cdate}.csv")
for k,g in tqdm.tqdm(df.sort_values(by="date", ascending=False).groupby("code"),
                         desc=f"processing date[{cdate}]"):
    changes = []
    values = dz.get(k[:2],dz["sh"])[-len(g):]
    g = g.sort_values(by="date",ascending=True).reset_index(drop=True)
    for i, r in g.iterrows():
        change = (g.iloc[i]["close"] - g.iloc[i-1]["close"])/g.iloc[i-1]["close"] if i else 0.0
        changes.append(change*100)
    y = np.array(changes).reshape((-1,1))
    reg = LinearRegression()
    reg.fit(values,y)
    preds = reg.predict(values)
    print(k,reg.coef_[0][0], reg.intercept_[0],pr)
    pr = pearsonr(preds.flatten(),y.flatten())[0]
    break

processing date[2025-01-02]:   0%|          | 0/4676 [00:00<?, ?it/s]

sh600000 0.06379983975877893 0.10628887187969888 0.050312362286398175
